# UA Q2 basic characterization

includes RR, ESR, Rabi, T1

## Header that we always run

In [1]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)
# setting CH02 to sweet spot
DC_ss = -1.196616
QDevil.setValue("CH02 Voltage", DC_ss)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 32
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
freq_TWPA = 6324E6;
pwr_TWPA = -10;
SG.setValue('Frequency', freq_TWPA)
SG.setValue('Power', pwr_TWPA)

client.close()

In [2]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from qm import SimulationConfig
import qdac as qdac
from scipy.io import savemat
from scipy.optimize import curve_fit
%matplotlib qt
import datetime 
import os
import time

qubit_index = 1 # for UA Q2

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)

2023-08-18 20:03:37,815 - qm - INFO     - Starting session: 1286e0fe-00f7-4c6e-b9ee-c253873cf857


In [3]:
# fitting functions
def ESR_Shape( x, amp, sigma, x0, c ):
    return amp * sigma**2 / ( sigma**2 + ( x - x0 )**2) + c

def Rabi_Shape( x, amp, f, phi, c):
    return amp * np.cos(2 * np.pi * f * x + phi) + c
def exp_decay( t, A, T1, c):
    return A*np.exp(-t/T1)+c

## RR spectroscopy

In [4]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'RR_spec'
qubit_name = 'UA_Q2'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

qubit_index = 1
res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo
n_avg = 10000
cooldown_time =20_000 # convert to ns in the wait function!

# The QUA program #
RR_freq_sweep = np.arange(-2e6, +2e6, 0.02e6)

with program() as resonator_spec:
    n = declare(int)
    n_st = declare_stream()
    df = declare(int)
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        save(n, n_st)
        with for_(*from_array(df, RR_freq_sweep)):
            update_frequency(machine.resonators[qubit_index].name, df + res_if)
            measure(
                "readout"*amp(1),
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        n_st.save('iteration')
        I_st.buffer(len(RR_freq_sweep)).average().save("I")
        Q_st.buffer(len(RR_freq_sweep)).average().save("Q")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

# Simulate or execute #
simulate = False # simulation is useful to see the sequence, especially the timing (clock cycle vs ns)

start_time = time.time()

if simulate:
    simulation_config = SimulationConfig(duration=1000)
    job = qmm.simulate(config, resonator_spec, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec)
    # Get results from QUA program
    res_handles = job.result_handles
    I_handles = res_handles.get("I")
    Q_handles = res_handles.get("Q")
    iteration_handles = res_handles.get("iteration")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [12, 8]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while job.result_handles.is_processing():
        # Fetch results
        time.sleep(0.1)
        iteration = res_handles.get("iteration").fetch_all()
        I = res_handles.get("I").fetch_all()
        Q = res_handles.get("Q").fetch_all()
        # progress bar
        progress_counter(iteration, n_avg, start_time = start_time)

2023-08-18 20:03:53,224 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [5]:
# Fetch results
iteration = res_handles.get("iteration").fetch_all()
I = res_handles.get("I").fetch_all()
Q = res_handles.get("Q").fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
# detrend removes the linear increase of phase
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
idx = np.argmin(sigs) # find minimum
print(f"IF offset to add to IF: {RR_freq_sweep[idx] / u.MHz:.6f} MHz")

# 1D spectroscopy plot
fig = plt.figure(figsize=[8, 4])
plt.title("Resonator spectroscopy")
plt.plot((machine.resonators[qubit_index].f_readout + RR_freq_sweep ) / u.MHz, sigs, ".")
plt.xlabel("Frequency [MHz]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
plt.axvline(x = (machine.resonators[qubit_index].f_readout + RR_freq_sweep[idx]) / u.MHz)

IF offset to add to IF: 0.160000 MHz


In [6]:
savemat(os.path.join(tPath, file_name), {"RR_freq": machine.resonators[qubit_index].f_readout + RR_freq_sweep, "sig": sigs, "sig_phase": phase, "DC_CH07": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [7]:
machine.resonators[qubit_index].f_readout += 0.16E6
machine._save("quam_state.json", flat_data=False)

## Flux sweep

In [ ]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'RR_spec_flux_sweep'
qubit_name = 'UA_Q2'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

In [ ]:
# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
# machine.resonators[0].f_readout+=1.68e6
# machine.resonators[0].readout_pulse_amp=0.27
config = build_config(machine)

# Program-specific variables #
n_avg =1000  # Number of averaging loops

cooldown_time = 10_000  # Resonator cooldown time, convert to ns in the wait() function 
flux_settle_time = 250  # Flux settle time for fast flux

flux_pts = 250
#dcs = np.linspace(-0.49, 0.49, flux_pts)
dc_flux_sweep = np.arange(-9,9 + 0.1,0.1)
RR_freq_sweep = np.arange(-7.5e6, 7.5e6, 0.05e6)

res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo

client = Labber.connectToServer('localhost') # get list of instruments 
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))

start_time = time.time()
for idx_dc, _ in enumerate(dc_flux_sweep):
    dc_flux = dc_flux_sweep[idx_dc]
    QDevil.setValue("CH02 Voltage", dc_flux)
# The QUA program #
    time.sleep(0.1)

    with program() as resonator_spec_2D:
        n = declare(int)  # Averaging index
        df = declare(int)  # Resonator frequency
        # dc = declare(fixed)  # flux dc level|
        I = declare(fixed)
        Q = declare(fixed)
        I_st = declare_stream()
        Q_st = declare_stream()
        n_st = declare_stream()
        #wait(flux_settle_time , machine.resonators[qubit_index].name, machine.qubits[qubit_index].name)

        with for_(n, 0, n < n_avg, n + 1):
            with for_(*from_array(df, RR_freq_sweep)):
                # Update the resonator frequency
                update_frequency(machine.resonators[qubit_index].name, df + res_if)
                # with for_(*from_array(dc, dcs)):
                    # Flux sweeping
                    # set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
                    # wait(flux_settle_time * u.ns, machine.resonators[qubit_index].name, machine.qubits[qubit_index].name)
                    # Measure the resonator
                measure(
                    "readout",
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("cos", "out1", "sin", "out2", I),
                    dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )
                # Wait for the resonator to cooldown
                wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
                # Save data to the stream processing
                save(I, I_st)
                save(Q, Q_st)
            save(n, n_st)

        with stream_processing():
            # I_st.buffer(len(dcs)).buffer(len(dfs)).average().save("I")
            # Q_st.buffer(len(dcs)).buffer(len(dfs)).average().save("Q")
            I_st.buffer(len(RR_freq_sweep)).average().save("I")
            Q_st.buffer(len(RR_freq_sweep)).average().save("Q")
            n_st.save("iteration")

    #  Open Communication with the QOP  #
    qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

    simulation = False

    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec_2D)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

    fig = plt.figure(1)
    plt.rcParams['figure.figsize'] = [8, 4]
    interrupt_on_close(fig, job)  #  Interrupts the job when closing the figure

    # Progress bar
    progress_counter(idx_dc, len(dc_flux_sweep), start_time=start_time)

    while results.is_processing():
        I_tmp, Q_tmp, iteration_tmp = results.fetch_all()
        I_tmp = u.demod2volts(I_tmp, machine.resonators[qubit_index].readout_pulse_length)
        Q_tmp = u.demod2volts(Q_tmp, machine.resonators[qubit_index].readout_pulse_length)

        # Live plotting
        plt.cla()
        plt.title("Resonator spectroscopy")
        plt.plot((machine.resonators[qubit_index].f_readout + RR_freq_sweep ) / u.MHz, np.sqrt(I_tmp**2 + Q_tmp**2), ".")
        plt.xlabel("Frequency [MHz]")
        plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
        plt.pause(0.1)
        interrupt_on_close(fig, job)  # Interrupts the job when closing the figure

    if idx_dc == 0:
        I_2D = I_tmp
        Q_2D = Q_tmp
    else:
        I_2D = np.hstack([I_2D, I_tmp])
        Q_2D = np.hstack([Q_2D, Q_tmp])

sigs = np.sqrt(I_2D**2 + Q_2D**2)
phase = signal.detrend(np.unwrap(np.angle(I_2D + 1j * Q_2D)))
savemat(os.path.join(tPath, file_name), {"RR_freq": machine.resonators[qubit_index].f_readout + RR_freq_sweep, "sig": sigs, "sig_phase": phase, "dc_flux_sweep": dc_flux_sweep, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [ ]:
client.close()

In [ ]:
I_2D = np.reshape(I_2D, (len(dc_flux_sweep), len(I_tmp)))
Q_2D = np.reshape(Q_2D, (len(dc_flux_sweep), len(Q_tmp)))
# 2D plot
fig = plt.figure()
plt.rcParams['figure.figsize'] = [8, 4]
# 2D spectroscopy plot
plt.cla()
plt.title("Resonator spectroscopy tuning curve")
plt.pcolor((machine.resonators[qubit_index].f_readout + RR_freq_sweep)/ u.MHz, dc_flux_sweep, np.sqrt(I_2D**2 + Q_2D**2), cmap="seismic")
plt.ylabel("DC flux[V]")
plt.xlabel("Frequency [MHz]")
plt.colorbar()

## ESR sweet spot

In [45]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'ESR'
qubit_name = 'UA_Q2'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

In [46]:
# set DC flux to sweet spot
client = Labber.connectToServer('localhost') # get list of instruments 
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))

QDevil.setValue("CH02 Voltage", DC_ss)
client.close()

In [66]:
# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

# Program-specific variables #
n_avg = 10000  # Number of averaging loops
cooldown_time = 20_000  # convert to ns in the wait() function
t = 80 * u.ns  # Qubit pulse length
freq_sweep = np.arange(-20e6, + 40e6, 0.5e6)

qubit_index = 1
qb_if = machine.qubits[qubit_index].f_01 - machine.qubits[qubit_index].lo

# The QUA program #
with program() as qubit_spec:
    n = declare(int)  # Averaging index
    df = declare(int)  # Resonator frequency
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    n_st = declare_stream()

    # Adjust the flux line if needed
    # set_dc_offset("flux_line", "single", 0.0)
    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(df, freq_sweep)):
            # Update the qubit frequency
            update_frequency(machine.qubits[qubit_index].name, df + qb_if)
            # Play a saturation pulse on the qubit
            # play("cw" * amp(0.05), machine.qubits[qubit_index].name, duration = t)
            play("x180" * amp(0.5), machine.qubits[qubit_index].name, duration = 4000)
            align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
            # Measure the resonator
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            # Wait for the resonator to cooldown
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
            # Save data to the stream processing
            save(I, I_st)
            save(Q, Q_st)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(freq_sweep)).average().save("I")
        Q_st.buffer(len(freq_sweep)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)
simulation = False

start_time = time.time()
if simulation:
    simulation_config = SimulationConfig(
        duration=8000, simulation_interface=LoopbackInterface([("con1", 3, "con1", 1)])
    )
    job = qmm.simulate(config, qubit_spec, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(qubit_spec)
    # Get results from QUA program
    res_handles = job.result_handles
    I_handles = res_handles.get("I")
    Q_handles = res_handles.get("Q")
    iteration_handles = res_handles.get("iteration")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [12, 8]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while job.result_handles.is_processing():
        # Fetch results
        time.sleep(0.1)
        iteration = res_handles.get("iteration").fetch_all()
        I = res_handles.get("I").fetch_all()
        Q = res_handles.get("Q").fetch_all()
        progress_counter(iteration, n_avg, start_time = start_time)

2023-08-19 07:27:13,213 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [67]:
# Fetch results
iteration = res_handles.get("iteration").fetch_all()
I = res_handles.get("I").fetch_all()
Q = res_handles.get("Q").fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
freqs = machine.qubits[qubit_index].f_01 + freq_sweep

In [68]:
# fit the data
try:
    param, dp_cov=curve_fit(ESR_Shape,freqs,sigs,p0=[max(sigs)-min(sigs),8E6,freqs[np.argmax(sigs)],min(sigs)] )
    d_ESR_freq = round(param[2] * 100)/100
    print('Updated f_01 (df) [MHz]:', param[2], (param[2] - machine.qubits[qubit_index].f_01) / u.MHz)
except:
    pass

# 1D spectroscopy plot
fig = plt.figure(figsize=[14, 6])
plt.plot(freqs / u.MHz, sigs, "b.")
plt.plot(freqs / u.MHz, ESR_Shape(freqs, param[0],param[1],param[2],param[3]), 'r')
plt.show()
plt.title("Qubit spectroscopy amplitude")

plt.xlabel("Frequency [MHz]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
plt.axvline(x=machine.qubits[qubit_index].f_01/u.MHz)

Updated f_01 (df) [MHz]: 6282213443.148338 -1.7418568516616821


In [69]:
savemat(os.path.join(tPath, file_name), {"ESR_freq": machine.qubits[qubit_index].f_01 + freq_sweep, "sig": sigs, "sig_phase": phase, "DC_flux": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [71]:
machine.qubits[qubit_index].f_01 += -1.74E6
machine._save("quam_state.json", flat_data=False)

## Rabi

In [72]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'Rabi'
qubit_name = 'UA_Q2'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

In [73]:
# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
# machine.qubits[0].pi_amp=0.109
# machine.qubits[0].pi_length=80
config = build_config(machine)

# The QUA program #
n_avg = 2_000
cooldown_time = 20_000 # ns, set in wait() function
amps = np.arange(0.1, 1.99, 0.01)
err_amp = 1  # Number of played qubit pulses for getting a better estimate of the pi amplitude
qubit_index = 1

with program() as power_rabi:
    n = declare(int)
    n_st = declare_stream()
    a = declare(fixed)
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(a, amps)):
            # Loop for error amplification (perform many qubit pulses)
            for i in range(err_amp):
                play("x180" * amp(a), machine.qubits[qubit_index].name)
            align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            save(I, I_st)
            save(Q, Q_st)
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(amps)).average().save("I")
        Q_st.buffer(len(amps)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

simulate = False
if simulate:
    simulation_config = SimulationConfig(duration=1000)  # in clock cycles
    job = qmm.simulate(config, power_rabi, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(power_rabi)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

    while results.is_processing():
        # Fetch results
        time.sleep(0.1)
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())

2023-08-19 07:30:52,173 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [74]:
I, Q, iteration = results.fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))

In [76]:
# fit the data
try:
    param, dp_cov=curve_fit(Rabi_Shape,amps,sigs,p0=[(max(sigs)-min(sigs))/2,1.2,0,(max(sigs)+min(sigs))/2] )
    amp_pi = round(((0.5-param[2]/2/np.pi)/param[1]) * 10000)/10000
    print('Updated amp_pi [V]:', amp_pi)
except:
    pass

# 1D spectroscopy plot
fig = plt.figure(figsize=[14, 6])
plt.plot(amps, sigs, "b.")
plt.plot(amps, Rabi_Shape(amps, param[0],param[1],param[2],param[3]), 'r')
plt.show()
plt.title("Rabi pulse amplitude [V]")

plt.xlabel("Amplitude [a.u.]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
plt.axvline(x=amp_pi)

Updated amp_pi [V]: 0.265


In [77]:
savemat(os.path.join(tPath, file_name), {"Rabi_amp": machine.qubits[qubit_index].pi_amp * amps, "sig": sigs, "sig_phase": phase, "DC_flux": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [78]:
#machine.qubits[qubit_index].pi_amp = machine.qubits[qubit_index].pi_amp * 0.67
machine._save("quam_state.json", flat_data=False)

## T1

In [86]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'T1'
qubit_name = 'UA_Q2'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

In [87]:
machine = QuAM("quam_state.json")
config = build_config(machine)

# The QUA program #
tau_min = 100 # clock cycle
tau_max = 8_000 # clock cycle
d_tau = 100 # clock cycle
tau_sweep = np.arange(tau_min, tau_max + 0.1, d_tau)  # + 0.1 to add t_max to taus
qubit_index = 1

n_avg = 10000
cooldown_time = 20_000 # ns

with program() as T1:
    n = declare(int)
    n_st = declare_stream()
    I = declare(fixed)
    I_st = declare_stream()
    Q = declare(fixed)
    Q_st = declare_stream()
    tau = declare(int)

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(tau, tau_sweep)):
            play("x180", machine.qubits[qubit_index].name)
            wait(tau, machine.qubits[qubit_index].name)
            align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("rotated_cos", "out1", "rotated_sin", "out2", I),
                dual_demod.full("rotated_minus_sin", "out1", "rotated_cos", "out2", Q),
            )
            save(I, I_st)
            save(Q, Q_st)
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(tau_sweep)).average().save("I")
        Q_st.buffer(len(tau_sweep)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

# Simulate or execute #
simulate = False
if simulate:
    simulation_config = SimulationConfig(duration=20_000)  # in clock cycles
    job = qmm.simulate(config, T1, simulation_config)
    job.get_simulated_samples().con1.plot()

else:
    print(datetime.datetime.now(),2)
    qm = qmm.open_qm(config)
    job = qm.execute(T1)
    
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [8, 6]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while results.is_processing():
        # Fetch results
        time.sleep(0.1)
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())

2023-08-19 07:36:04,064 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [88]:
# Fetch results
I, Q, iteration = results.fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
taus = tau_sweep * 4.0 * u.ns # convert clock cycle to ns

In [89]:
try:
    param, _ = curve_fit(exp_decay,taus,sigs,p0=[max(sigs)-min(sigs),4E3,min(sigs)] )
    q_T1 = round(param[1])
    print('Qubit T1 [us]:', q_T1 /u.us)
except:
    pass

# 1D spectroscopy plot
fig = plt.figure(figsize=[14, 6])
plt.plot(taus /u.us, sigs, "b.")
plt.plot(taus / u.us, exp_decay(taus, param[0],param[1],param[2]), 'r')
plt.show()
plt.title("Qubit T1")
plt.xlabel("tau [us]")
plt.ylabel("$\sqrt{I^2 + Q^2}$ [V]")

Qubit T1 [us]: 5.984


Text(0, 0.5, '$\\sqrt{I^2 + Q^2}$ [V]')

In [90]:
savemat(os.path.join(tPath, file_name), {"tau": taus, "sig": sigs, "sig_phase": phase, "DC_flux": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [91]:
machine.qubits[qubit_index].T1 = q_T1
machine._save("quam_state.json", flat_data=False)

## Qubit spectroscopy

In [ ]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'Qubit_spec'
qubit_name = 'UA_Q2'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

In [ ]:
# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

# Program-specific variables #
n_avg = 1000  # Number of averaging loops
cooldown_time = 20_000  # convert to ns in the wait() function
freq_sweep = np.arange(-100e6, + 40e6, 1e6)
ff_sweep = np.arange(-0.5, 0.0, 0.05)
flux_settle_time = 10

qubit_index = 1
qb_if = machine.qubits[qubit_index].f_01 - machine.qubits[qubit_index].lo

# The QUA program #
with program() as qubit_spec:
    n = declare(int)  # Averaging index
    df = declare(int)  # IF frequency
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    n_st = declare_stream()

    # Adjust the flux line if needed
    # set_dc_offset("flux_line", "single", 0.0)
    with for_(n, 0, n < n_avg, n + 1):
        save(n, n_st)
        with for_(*from_array(ff, ff_sweep))
            with for_(*from_array(df, freq_sweep)):
                # Update the qubit frequency
                update_frequency(machine.qubits[qubit_index].name, df + qb_if)
                # fast flux, pi pulse, turn off fast flux (through a manual dc plateau)
                set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
                wait(flux_settle_time, machine.flux_lines[qubit_index].name)
                align()
                play("x180" * amp(1.0), machine.qubits[qubit_index].name)
                wait(flux_settle_time, machine.flux_lines[qubit_index].name)
                align()
                set_dc_offset(machine.flux_lines[qubit_index].name, "single", machine.flux_lines[qubit_index].max_frequency_point)
                wait(10)
                align()
                
                
                align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
                # Measure the resonator
                measure(
                    "readout",
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("cos", "out1", "sin", "out2", I),
                    dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )
                # Wait for the resonator to cooldown
                wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
                # Save data to the stream processing
                save(I, I_st)
                save(Q, Q_st)
        

    with stream_processing():
        I_st.buffer(len(freq_sweep)).average().save("I")
        Q_st.buffer(len(freq_sweep)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)
simulation = False

start_time = time.time()
if simulation:
    simulation_config = SimulationConfig(
        duration=8000, simulation_interface=LoopbackInterface([("con1", 3, "con1", 1)])
    )
    job = qmm.simulate(config, qubit_spec, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(qubit_spec)
    # Get results from QUA program
    res_handles = job.result_handles
    I_handles = res_handles.get("I")
    Q_handles = res_handles.get("Q")
    iteration_handles = res_handles.get("iteration")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [12, 8]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while job.result_handles.is_processing():
        # Fetch results
        time.sleep(0.1)
        iteration = res_handles.get("iteration").fetch_all()
        I = res_handles.get("I").fetch_all()
        Q = res_handles.get("Q").fetch_all()
        progress_counter(iteration, n_avg, start_time = start_time)

In [ ]:
play("x180", machine.qubits[qubit_index].name)
                align()
                set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
                wait(t, machine.flux_lines[qubit_index].name)
                align()
                set_dc_offset(machine.flux_lines[qubit_index].name, "single", machine.flux_lines[qubit_index].max_frequency_point)
                wait(10)
                align()
                measure(
                    "readout",
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("rotated_cos", "out1", "rotated_sin", "out2", I),
                    dual_demod.full("rotated_minus_sin", "out1", "rotated_cos", "out2", Q),
                )
                save(I, I_st)
                save(Q, Q_st)
                wait(cooldown_time, machine.resonators[qubit_index].name)